In [ ]:
# Блок 1: Введение в Обнаружение Объектов (Object Detection)

# Задача: Не только классифицировать объекты на изображении, но и определить
# их точное местоположение с помощью ограничивающих рамок (bounding boxes).
# Вход: Изображение.
# Выход: Список обнаруженных объектов, каждый с рамкой, меткой класса и уверенностью.

# Модель: Будем использовать Faster R-CNN с backbone ResNet-50 FPN,
# предобученную на COCO, и дообучим её на простом синтетическом датасете.
# Faster R-CNN - это двухэтапный детектор:
# 1. Region Proposal Network (RPN): Генерирует "предложения" регионов, где могут быть объекты.
# 2. Классификатор и Регрессор Рамок: Классифицирует объекты в предложенных регионах
#    и уточняет координаты их рамок.

# Датасет: Создадим синтетический датасет с цветными квадратами и кругами на белом фоне.
# Классы:
# 0: __background__ (фон - обязателен для моделей torchvision)
# 1: red_square
# 2: blue_circle

# --------------------------------------------------

# Блок 2: Импорты и Настройки

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms.functional import to_tensor # Для конвертации PIL в тензор
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import os
import copy
from tqdm import tqdm # Индикатор прогресса

# Настройки
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {DEVICE}")

# Параметры датасета и модели
NUM_CLASSES = 3 # 2 наших класса + 1 фон
IMG_SIZE = 256 # Размер генерируемых изображений

# Параметры обучения
BATCH_SIZE = 4 # Уменьшите, если не хватает памяти GPU
NUM_EPOCHS = 10 # Для примера, в реальности нужно больше
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0005
MODEL_SAVE_PATH = "fasterrcnn_shapes_best.pth"

# --------------------------------------------------

# Блок 3: Создание Синтетического Датасета

class ShapesDataset(Dataset):
    """
    Генерирует синтетические изображения с красными квадратами и синими кругами.
    """
    def __init__(self, num_samples=200, img_size=256, transform=None):
        self.num_samples = num_samples
        self.img_size = img_size
        self.transform = transform # Пока не используем сложные трансформации
        self.images = []
        self.targets = []
        self._generate_data()

    def _generate_data(self):
        print(f"Generating {self.num_samples} synthetic images...")
        for idx in range(self.num_samples):
            # Создаем белое изображение
            img = Image.new('RGB', (self.img_size, self.img_size), color='white')
            draw = ImageDraw.Draw(img)
            boxes = []
            labels = []

            # Добавляем 1-4 фигуры
            num_shapes = np.random.randint(1, 5)
            for _ in range(num_shapes):
                shape_type = np.random.choice(['square', 'circle'])
                size = np.random.randint(25, 70) # Размер фигуры
                # Случайные координаты (с отступом от края)
                x = np.random.randint(10, self.img_size - size - 10)
                y = np.random.randint(10, self.img_size - size - 10)
                xmin, ymin, xmax, ymax = x, y, x + size, y + size

                # Проверка на пересечение с уже добавленными фигурами (упрощенная)
                overlaps = False
                for existing_box in boxes:
                    # Простая проверка пересечения по x и y
                    if not (xmax < existing_box[0] or xmin > existing_box[2] or
                            ymax < existing_box[1] or ymin > existing_box[3]):
                        overlaps = True
                        break
                if overlaps:
                    continue # Пропускаем эту фигуру, если есть пересечение

                if shape_type == 'square':
                    color = 'red'
                    label = 1 # Метка для красного квадрата
                    draw.rectangle([xmin, ymin, xmax, ymax], fill=color, outline='black')
                else: # circle
                    color = 'blue'
                    label = 2 # Метка для синего круга
                    draw.ellipse([xmin, ymin, xmax, ymax], fill=color, outline='black')

                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(label)

            # Если на изображении не оказалось фигур (из-за пересечений), пропускаем его
            if not boxes:
                continue

            self.images.append(img)
            target = {}
            # Конвертируем в тензоры нужного типа
            target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
            target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
            target["image_id"] = torch.tensor([idx]) # Уникальный ID изображения
            # Рассчитываем площадь рамок
            target["area"] = (target["boxes"][:, 3] - target["boxes"][:, 1]) * \
                             (target["boxes"][:, 2] - target["boxes"][:, 0])
            # iscrowd=0 означает, что рамки не являются группами объектов (стандартно для своих данных)
            target["iscrowd"] = torch.zeros((len(boxes),), dtype=torch.int64)
            self.targets.append(target)
        print(f"Generated {len(self.images)} valid images.")


    def __getitem__(self, idx):
        img = self.images[idx]
        target = self.targets[idx]

        # Применяем базовую трансформацию (конвертация в тензор)
        # В реальной задаче здесь были бы аугментации (Albumentations)
        img_tensor = to_tensor(img)

        # if self.transform:
        #     # Применить трансформации к img и target['boxes']
        #     # Важно: трансформации должны корректно обрабатывать и рамки!
        #     pass

        return img_tensor, target

    def __len__(self):
        # Возвращаем количество успешно сгенерированных изображений
        return len(self.images)

# --- 3.1 Функция collate_fn для DataLoader ---
# Необходима, т.к. таргеты (словари) для разных изображений имеют разный размер
# (разное количество объектов) и не могут быть просто сложены в один тензор.
def collate_fn(batch):
    # batch - это список кортежей [(img1, target1), (img2, target2), ...]
    # Функция list(zip(*batch)) преобразует его в ([img1, img2, ...], [target1, target2, ...])
    return tuple(zip(*batch))

# --- 3.2 Создание Датасетов и Загрузчиков ---
# Создаем датасеты
train_dataset = ShapesDataset(num_samples=250, img_size=IMG_SIZE) # Больше данных для обучения
val_dataset = ShapesDataset(num_samples=50, img_size=IMG_SIZE)   # Меньше для валидации

# Создаем загрузчики
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate_fn, num_workers=0) # num_workers=0 для Windows/простоты
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        collate_fn=collate_fn, num_workers=0)

print(f"\nDataLoaders created. Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

# --------------------------------------------------

# Блок 4: Определение Модели Faster R-CNN

def get_object_detection_model(num_classes):
    # Загружаем предобученную модель Faster R-CNN с ResNet-50 FPN backbone
    # Используем новые веса API (рекомендуется)
    weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

    # Получаем количество входных признаков для классификатора
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # Заменяем предобученную голову классификатора на новую
    # num_classes включает класс фона (__background__)
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # --- Опционально: Замена генератора якорей (если стандартные не подходят) ---
    # anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
    #                                    aspect_ratios=((0.5, 1.0, 2.0),))
    # model.rpn.anchor_generator = anchor_generator

    return model

# Инициализация модели
model = get_object_detection_model(NUM_CLASSES)
model.to(DEVICE)
print("\nFaster R-CNN model loaded and modified for custom classes.")

# --------------------------------------------------

# Блок 5: Настройка Обучения (Оптимизатор, Планировщик)

# Выбираем параметры, которые требуют обновления градиентов
params = [p for p in model.parameters() if p.requires_grad]

# Оптимизатор (SGD часто рекомендуется для fine-tuning детекторов)
optimizer = torch.optim.SGD(params, lr=LEARNING_RATE, momentum=0.9, weight_decay=WEIGHT_DECAY)

# Планировщик скорости обучения (уменьшает LR со временем)
# Уменьшаем LR в 10 раз каждые 3 эпохи
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

print("Optimizer and LR Scheduler configured.")

# --------------------------------------------------

# Блок 6: Цикл Обучения и Валидации

print("\nStarting Training...")
training_start_time = time.time()
best_val_loss = float('inf') # Для сохранения лучшей модели
train_loss_history = []
val_loss_history = []

for epoch in range(NUM_EPOCHS):
    # --- Фаза Обучения ---
    model.train() # Переводим модель в режим обучения
    epoch_train_loss = 0
    pbar_train = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Training]")

    for images, targets in pbar_train:
        # Перемещаем данные на устройство
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        # Прямой проход -> модель возвращает словарь лоссов в режиме train
        loss_dict = model(images, targets)
        # Суммируем все лоссы (loss_classifier, loss_box_reg, loss_objectness, loss_rpn_box_reg)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item() # Получаем скалярное значение лосса
        epoch_train_loss += loss_value

        # Обратный проход и оптимизация
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        # Обновляем прогресс-бар
        pbar_train.set_postfix({'Loss': loss_value})

    avg_epoch_train_loss = epoch_train_loss / len(train_loader)
    train_loss_history.append(avg_epoch_train_loss)
    print(f"Epoch {epoch+1} Train Summary: Avg Loss: {avg_epoch_train_loss:.4f}")

    # --- Фаза Валидации ---
    model.eval() # Переводим модель в режим оценки
    epoch_val_loss = 0
    pbar_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Validation]")

    with torch.no_grad(): # Отключаем вычисление градиентов
        for images, targets in pbar_val:
            images = list(image.to(DEVICE) for image in images)
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

            # В режиме eval модель тоже может вернуть лоссы, если передать таргеты
            # Это удобно для мониторинга валидационного лосса
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()
            epoch_val_loss += loss_value
            pbar_val.set_postfix({'Loss': loss_value})

    avg_epoch_val_loss = epoch_val_loss / len(val_loader)
    val_loss_history.append(avg_epoch_val_loss)
    print(f"Epoch {epoch+1} Validation Summary: Avg Loss: {avg_epoch_val_loss:.4f}")

    # Обновляем планировщик LR
    lr_scheduler.step()

    # Сохраняем лучшую модель по валидационному лоссу
    if avg_epoch_val_loss < best_val_loss:
        best_val_loss = avg_epoch_val_loss
        # Сохраняем только state_dict для экономии места
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"Checkpoint saved: Improved validation loss to {best_val_loss:.4f}")

training_end_time = time.time()
print(f"\nTraining finished in {(training_end_time - training_start_time)/60:.2f} minutes.")
print(f"Best validation loss achieved: {best_val_loss:.4f}")

# --- 6.1 Визуализация Лоссов Обучения ---
# plt.figure(figsize=(10, 5))
# plt.plot(range(1, NUM_EPOCHS + 1), train_loss_history, label='Training Loss')
# plt.plot(range(1, NUM_EPOCHS + 1), val_loss_history, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss Over Epochs')
# plt.legend()
# plt.grid(True)
# plt.show()

# --------------------------------------------------

# Блок 7: Инференс и Визуализация Результатов

print("\nLoading best model for inference...")
# Загружаем лучшую модель (сохраненный state_dict)
# Сначала нужно создать экземпляр модели той же архитектуры
inference_model = get_object_detection_model(NUM_CLASSES)
# Загружаем веса
try:
    inference_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
    inference_model.to(DEVICE)
    inference_model.eval() # Переводим в режим оценки
    print("Best model loaded successfully.")
except FileNotFoundError:
    print(f"Error: Saved model file not found at {MODEL_SAVE_PATH}. Using the last state of the model.")
    # Если файл не найден, используем модель после последней эпохи (уже в режиме eval)
    inference_model = model # model уже на DEVICE и в eval() после валидации
except Exception as e:
     print(f"An error occurred loading the model: {e}. Using the last state of the model.")
     inference_model = model


print("\nRunning Inference on a sample validation image...")

# Словарь для имен классов (для визуализации)
CLASS_NAMES = {0: 'background', 1: 'red_square', 2: 'blue_circle'}

# Функция для отрисовки
def plot_predictions(img_tensor, prediction, true_target=None, threshold=0.5):
    # Переводим тензор обратно в PIL Image для отрисовки
    img_pil = torchvision.transforms.ToPILImage()(img_tensor.cpu())
    draw = ImageDraw.Draw(img_pil)
    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.imshow(img_pil)
    ax.set_title("Model Prediction vs Ground Truth")
    ax.axis('off')

    # Истинные рамки (зеленые)
    if true_target:
        true_boxes = true_target['boxes'].cpu().numpy()
        true_labels = true_target['labels'].cpu().numpy()
        for box, label_idx in zip(true_boxes, true_labels):
            xmin, ymin, xmax, ymax = box
            label_name = CLASS_NAMES.get(label_idx, 'unknown')
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                     linewidth=2, edgecolor='g', facecolor='none')
            ax.add_patch(rect)
            ax.text(xmin, ymin - 10, f"True: {label_name}", color='green', fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.6, pad=0.1, edgecolor='none'))

    # Предсказанные рамки (красные)
    pred_boxes = prediction['boxes'].cpu().numpy()
    pred_labels = prediction['labels'].cpu().numpy()
    pred_scores = prediction['scores'].cpu().numpy()

    for box, label_idx, score in zip(pred_boxes, pred_labels, pred_scores):
        if score >= threshold:
            xmin, ymin, xmax, ymax = box
            label_name = CLASS_NAMES.get(label_idx, 'unknown')
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                     linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(xmin, ymax + 5, f"Pred: {label_name} ({score:.2f})", color='red', fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.6, pad=0.1, edgecolor='none'))

    plt.show()


# Получаем случайный пример из валидационного набора
img_idx = np.random.randint(len(val_dataset))
img_tensor, target = val_dataset[img_idx]

# Запускаем инференс
with torch.no_grad():
    # Модель ожидает список изображений, даже если оно одно
    prediction = inference_model([img_tensor.to(DEVICE)])[0] # Берем предсказания для первого (и единственного) изображения

# Визуализируем результат
plot_predictions(img_tensor, prediction, true_target=target, threshold=0.5)

print("Inference and visualization complete for one sample.")

# --- Конец Примера ---

# --------------------------------------------------

In [ ]:
# Блок 1: Задача Продолжения Текста (Text Continuation / Next Word Prediction)

# Что это за Задача?
# Это задача NLP, где модель учится предсказывать следующее слово (или символ)
# в последовательности, основываясь на предыдущих словах (контексте).
# Это фундаментальная задача Языкового Моделирования (Language Modeling - LM).
# Вход: Последовательность слов (например, "the quick brown fox").
# Выход: Вероятностное распределение по словарю для следующего слова (например,
#        высокая вероятность для "jumps", низкая для "apple").

# Применение:
# - Автодополнение текста.
# - Генерация текста (стихи, код, истории).
# - Основа для более сложных моделей (например, в машинном переводе).

# Почему LSTM?
# Long Short-Term Memory (LSTM) - это тип Рекуррентной Нейронной Сети (RNN),
# который хорошо подходит для обработки последовательных данных, таких как текст.
# LSTM способны "запоминать" информацию на длинных дистанциях в последовательности
# благодаря своим внутренним механизмам (гейтам), что помогает им улавливать
# контекст и предсказывать следующее слово более осмысленно, чем простые RNN или N-граммы.

# Подход:
# Мы обучим LSTM-модель предсказывать следующее слово в последовательности.
# Модель будет принимать на вход последовательность слов и выдавать
# вероятности для каждого слова в словаре как кандидата на следующее слово.

# --------------------------------------------------

# Блок 2: Импорты и Настройки

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from tqdm import tqdm
import numpy as np
import re
import random

# Настройки
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Параметры модели и обучения
EMBED_DIM = 128      # Размерность эмбеддингов слов
HIDDEN_DIM = 256     # Размерность скрытого состояния LSTM
NUM_LAYERS = 2       # Количество слоев LSTM
DROPOUT_PROB = 0.4
LEARNING_RATE = 0.002
NUM_EPOCHS = 50      # Для языковых моделей нужно больше эпох
BATCH_SIZE = 32
SEQ_LENGTH = 20      # Длина входной последовательности для модели
VOCAB_SIZE = 0       # Определится после построения словаря

# Специальные токены
PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>" # Не используется в этом простом примере, т.к. словарь строится по всем данным

# --------------------------------------------------

# Блок 3: Подготовка Данных

# --- 3.1 Пример Текстовых Данных ---
# Используем простой текст для демонстрации. В реальной задаче нужен большой корпус.
text_data = """
Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence
concerned with the interactions between computers and human language, in particular how to program computers
to process and analyze large amounts of natural language data. The goal is a computer capable of
understanding the content of documents, including the contextual nuances of the language within them.
The technology can then accurately extract information and insights contained in the documents as well as
categorize and organize the documents themselves. Challenges in natural language processing frequently
involve speech recognition, natural language understanding, and natural language generation.
Language models based on deep learning architectures like recurrent neural networks (RNN) and transformers
have achieved state-of-the-art results on many NLP tasks.
"""

# --- 3.2 Предобработка и Токенизация ---
def preprocess_and_tokenize(text):
    text = text.lower()
    # Заменяем переносы строк и множественные пробелы на один пробел
    text = re.sub(r'\s+', ' ', text).strip()
    # Простая токенизация по пробелам (можно использовать nltk или spaCy для лучшей токенизации)
    tokens = text.split(' ')
    # Удаляем пустые токены, если они появились
    tokens = [token for token in tokens if token]
    return tokens

tokens = preprocess_and_tokenize(text_data)
# print(f"Total tokens: {len(tokens)}")
# print(f"Sample tokens: {tokens[:20]}")

# --- 3.3 Построение Словаря ---
word_counts = Counter(tokens)
# Создаем словарь: слово -> индекс и обратный словарь: индекс -> слово
# Не добавляем UNK, т.к. все слова из текста будут в словаре
vocab = {word: i+1 for i, (word, count) in enumerate(word_counts.items())} # Начинаем с 1, 0 для PAD
vocab[PAD_TOKEN] = 0
idx_to_vocab = {i: word for word, i in vocab.items()}
VOCAB_SIZE = len(vocab)
print(f"Vocabulary size: {VOCAB_SIZE}")

# --- 3.4 Создание Последовательностей ---
# Создаем входные последовательности (X) и целевые последовательности (y)
# X: последовательность длины SEQ_LENGTH
# y: следующее слово после последовательности X
sequences = []
for i in range(len(tokens) - SEQ_LENGTH):
    input_seq = tokens[i : i + SEQ_LENGTH]
    target_word = tokens[i + SEQ_LENGTH]
    sequences.append((input_seq, target_word))

# print(f"\nNumber of sequences created: {len(sequences)}")
# print(f"Example sequence:")
# print(f"  Input (X): {' '.join(sequences[0][0])}")
# print(f"  Target (y): {sequences[0][1]}")

# --- 3.5 Конвертация в Индексы ---
def sequence_to_indices(seq_tuple, vocab):
    input_indices = [vocab.get(token, -1) for token in seq_tuple[0]] # -1 для отладки, если слово не найдено
    target_index = vocab.get(seq_tuple[1], -1)
    # Проверка на случай, если слово не нашлось (не должно произойти с этим словарем)
    if -1 in input_indices or target_index == -1:
        print("Warning: Token not found in vocab during index conversion!")
        # Простая обработка - пропустить последовательность
        return None, None
    return input_indices, target_index

indexed_sequences = []
for seq_in, seq_out in sequences:
    idx_in, idx_out = sequence_to_indices((seq_in, seq_out), vocab)
    if idx_in is not None:
        indexed_sequences.append((idx_in, idx_out))

# print(f"\nNumber of indexed sequences: {len(indexed_sequences)}")
# print(f"Example indexed sequence:")
# print(f"  Input (X indices): {indexed_sequences[0][0]}")
# print(f"  Target (y index): {indexed_sequences[0][1]}")

# --- 3.6 Создание PyTorch Dataset ---
class TextGenerationDataset(Dataset):
    def __init__(self, indexed_sequences):
        self.indexed_sequences = indexed_sequences

    def __len__(self):
        return len(self.indexed_sequences)

    def __getitem__(self, idx):
        input_seq, target_idx = self.indexed_sequences[idx]
        # Возвращаем тензоры
        # Input должен быть Long для Embedding слоя
        # Target должен быть Long для CrossEntropyLoss
        return torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_idx, dtype=torch.long)

dataset = TextGenerationDataset(indexed_sequences)

# --- 3.7 Создание DataLoader ---
# Паддинг не нужен, т.к. все последовательности имеют одинаковую длину SEQ_LENGTH
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# --------------------------------------------------

# Блок 4: Определение Модели (LSTM)

class LSTMNextWordPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, dropout_prob):
        super(LSTMNextWordPredictor, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim,
                            hidden_dim,
                            num_layers=num_layers,
                            batch_first=True, # Ожидаем вход [batch, seq, feature]
                            dropout=dropout_prob if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout_prob)
        # Линейный слой для предсказания следующего слова по всему словарю
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, text_indices, hidden):
        # text_indices: [batch_size, seq_len]
        # hidden: tuple (h_n, c_n)
        #   h_n: [num_layers * num_directions, batch_size, hidden_dim]
        #   c_n: [num_layers * num_directions, batch_size, hidden_dim]

        embedded = self.embedding(text_indices)
        # embedded: [batch_size, seq_len, embed_dim]

        lstm_out, hidden = self.lstm(embedded, hidden)
        # lstm_out: [batch_size, seq_len, hidden_dim]
        # hidden: tuple (h_n, c_n) - обновленные состояния

        # Мы хотим предсказать слово ПОСЛЕ последнего слова во входной последовательности,
        # поэтому используем выход LSTM на последнем временном шаге.
        last_step_output = lstm_out[:, -1, :] # [batch_size, hidden_dim]

        # Применяем Dropout и полносвязный слой
        out = self.dropout(last_step_output)
        logits = self.fc(out)
        # logits: [batch_size, vocab_size]
        return logits, hidden

    def init_hidden(self, batch_size, device):
        # Инициализация скрытого состояния и состояния ячейки нулями
        weight = next(self.parameters()).data
        # Умножаем на 1, т.к. у нас нет bidirectional
        hidden = (weight.new(self.num_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.num_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

# Инициализация модели
model = LSTMNextWordPredictor(
    vocab_size=VOCAB_SIZE,
    embed_dim=EMBED_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    dropout_prob=DROPOUT_PROB
)
model.to(DEVICE)
print("\nLSTM Model Initialized:")
print(model)
print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

# --------------------------------------------------

# Блок 5: Обучение Модели

criterion = nn.CrossEntropyLoss() # Подходит для предсказания индекса следующего слова
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("\nStarting Training...")
training_start_time = time.time()

model.train() # Установить режим обучения

for epoch in range(NUM_EPOCHS):
    # Инициализация скрытого состояния для каждой эпохи (или батча, если stateful)
    # Для stateless RNN, инициализируем для каждого батча
    epoch_loss = 0
    num_batches = 0

    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for input_seqs, target_words in pbar:
        num_batches += 1
        # Инициализация скрытого состояния для батча
        h = model.init_hidden(input_seqs.size(0), DEVICE) # input_seqs.size(0) == batch_size

        # Перемещаем данные на устройство
        input_seqs = input_seqs.to(DEVICE)
        target_words = target_words.to(DEVICE) # target_words: [batch_size]

        # Отсоединяем скрытые состояния от истории вычислений предыдущего батча
        h = tuple([each.data for each in h])

        optimizer.zero_grad()
        # Получаем логиты и новые скрытые состояния
        logits, h = model(input_seqs, h)
        # logits: [batch_size, vocab_size]

        # Считаем лосс
        loss = criterion(logits, target_words)
        loss.backward()

        # Опционально: обрезка градиента для предотвращения взрыва
        nn.utils.clip_grad_norm_(model.parameters(), 5)

        optimizer.step()

        epoch_loss += loss.item()
        pbar.set_postfix({'Loss': loss.item()})

    avg_epoch_loss = epoch_loss / num_batches
    print(f"Epoch {epoch+1} Summary: Average Loss: {avg_epoch_loss:.4f}")

training_end_time = time.time()
print(f"\nTraining finished in {(training_end_time - training_start_time)/60:.2f} minutes.")

# --------------------------------------------------

# Блок 6: Генерация Текста (Инференс)

def generate_text(model, tokenizer_vocab, idx_to_vocab, seed_text, n_words_to_generate, device, temperature=1.0):
    """Генерирует текст, начиная с seed_text."""
    model.eval() # Переводим модель в режим оценки

    # Предобработка seed_text
    tokens = preprocess_and_tokenize(seed_text)
    # print(f"Seed tokens: {tokens}")

    # Инициализация скрытого состояния
    # Начинаем с батча размером 1
    h = model.init_hidden(1, device)

    generated_words = tokens.copy() # Начинаем генерацию с исходных токенов

    # "Прогреваем" модель на seed_text, чтобы получить начальное скрытое состояние
    # соответствующее концу seed_text
    if len(tokens) > 0:
        seed_indices = [tokenizer_vocab.get(token, -1) for token in tokens]
        # Проверка на неизвестные слова в seed_text
        if -1 in seed_indices:
             print(f"Warning: Seed text contains words not in vocabulary: {[tokens[i] for i, idx in enumerate(seed_indices) if idx == -1]}")
             # Можно заменить на UNK, если он есть, или просто проигнорировать
             seed_indices = [idx if idx != -1 else 0 for idx in seed_indices] # Заменяем на PAD для простоты

        seed_tensor = torch.tensor(seed_indices).unsqueeze(0).to(device) # [1, seed_len]
        # Прогоняем весь seed через модель, чтобы получить последнее скрытое состояние
        with torch.no_grad():
            _, h = model(seed_tensor, h)
        # Используем последний токен из seed как вход для первого предсказания
        current_input_idx = seed_indices[-1]
    else:
        # Если seed пустой, начинаем с PAD токена или случайного
        current_input_idx = tokenizer_vocab[PAD_TOKEN]


    # Генерация следующих n_words_to_generate слов
    for _ in range(n_words_to_generate):
        # Подготавливаем входной тензор (только последний предсказанный токен)
        input_tensor = torch.tensor([[current_input_idx]], dtype=torch.long).to(device) # [1, 1]

        # Получаем предсказание и обновляем скрытое состояние
        with torch.no_grad():
            logits, h = model(input_tensor, h) # h передается и обновляется

        # Применяем температуру к логитам перед softmax
        # Температура < 1.0 делает распределение более "пиковым" (уверенным)
        # Температура > 1.0 делает распределение более "плоским" (случайным)
        logits = logits.squeeze(0) / temperature

        # Получаем вероятности
        probabilities = torch.softmax(logits, dim=-1)

        # Сэмплируем следующее слово на основе вероятностей
        # multinomial ожидает 1D тензор вероятностей
        next_word_idx = torch.multinomial(probabilities, num_samples=1).item()

        # Если предсказан PAD, останавливаемся (или пробуем снова)
        if next_word_idx == tokenizer_vocab[PAD_TOKEN]:
            continue # Пропускаем PAD

        # Декодируем индекс в слово
        next_word = idx_to_vocab.get(next_word_idx, UNK_TOKEN) # Используем UNK, если индекс некорректен

        generated_words.append(next_word)
        # Обновляем вход для следующего шага
        current_input_idx = next_word_idx

    return " ".join(generated_words)

# --- Примеры Генерации ---
print("\n--- Text Generation Examples ---")

seed1 = "natural language"
generated1 = generate_text(model, vocab, idx_to_vocab, seed1, 30, DEVICE, temperature=0.8)
print(f"Seed: '{seed1}'")
print(f"Generated: '{generated1}'\n")

seed2 = "deep learning architectures like"
generated2 = generate_text(model, vocab, idx_to_vocab, seed2, 25, DEVICE, temperature=1.0)
print(f"Seed: '{seed2}'")
print(f"Generated: '{generated2}'\n")

seed3 = "the goal is a computer"
generated3 = generate_text(model, vocab, idx_to_vocab, seed3, 20, DEVICE, temperature=0.5)
print(f"Seed: '{seed3}'")
print(f"Generated: '{generated3}'\n")

# --- Конец Примера ---

# --------------------------------------------------

# Блок 7: Ограничения и Улучшения

# Ограничения Этого Примера:
# - **Маленький Датасет:** Модель обучалась на очень маленьком тексте, поэтому
#   ее знания о языке крайне ограничены. Генерация будет повторяющейся и не очень осмысленной.
# - **Простая Токенизация:** Разделение по пробелам не обрабатывает пунктуацию и сложные случаи.
# - **Фиксированная Длина Контекста:** LSTM видит только `SEQ_LENGTH` предыдущих слов.
# - **Простая Генерация:** Сэмплирование по вероятностям может приводить к повторам.
#   Более сложные методы (Top-k sampling, Top-p (nucleus) sampling) дают лучшие результаты.
# - **Нет Обработки UNK:** Неизвестные слова в seed_text обрабатываются примитивно.

# Возможные Улучшения:
# - **Больше Данных:** Обучение на большом корпусе (книги, статьи).
# - **Лучшая Токенизация:** Использовать `spaCy` или `nltk` или токенизаторы подслов (BPE, WordPiece).
# - **Увеличение Модели:** Больше `EMBED_DIM`, `HIDDEN_DIM`, `NUM_LAYERS`.
# - **Более Длинный Контекст:** Увеличить `SEQ_LENGTH` (требует больше памяти).
# - **Продвинутые Методы Сэмплирования:** Top-k, Top-p.
# - **Внимание (Attention):** Добавление механизма внимания может улучшить обработку длинных зависимостей.
# - **Использование Трансформеров:** Модели типа GPT специально разработаны для этой задачи и дают state-of-the-art результаты (но требуют больше ресурсов).

# --------------------------------------------------

In [ ]:
# Блок 1: Введение в Задачу и Инструменты

# Задача: Линейная Регрессия с Помощью SVM (SVR)
# Цель: Предсказать непрерывное числовое значение (Y), используя линейную
#       зависимость от входных признаков (X).
# Модель: Support Vector Regressor (SVR) с линейным ядром (`kernel='linear'`).
#       SVR пытается найти гиперплоскость, которая наилучшим образом аппроксимирует
#       данные, оставляя максимальное количество точек внутри "трубки" (margin)
#       шириной 2*epsilon, и минимизируя ошибки для точек вне трубки.
# Предобработка: MinMaxScaler для масштабирования признаков в диапазон [0, 1].
#       Это **критически важно** для SVR, так как он чувствителен к масштабу.
# Постобработка: Обратное преобразование предсказанных значений в исходный масштаб
#       для интерпретации и оценки.

# --------------------------------------------------

# Блок 2: Импорты и Настройки

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR # Support Vector Regressor
from sklearn.preprocessing import MinMaxScaler # Для масштабирования
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Настройки (можно менять)
RANDOM_STATE = 42
TEST_SIZE = 0.25
SVR_C = 1.0       # Параметр регуляризации SVR
SVR_EPSILON = 0.1 # Ширина "трубки" SVR

# --------------------------------------------------

# Блок 3: Генерация и Подготовка Данных

# --- 3.1 Генерация Синтетических Данных ---
# Создадим данные с примерно линейной зависимостью + шум
np.random.seed(RANDOM_STATE)
X = 2 * np.random.rand(100, 1) # 100 примеров, 1 признак (от 0 до 2)
# Истинная зависимость y = 5 + 2*X + шум
y = 5 + 2 * X + np.random.randn(100, 1) * 0.8 # Добавим немного шума

# print("Shape of X:", X.shape) # (100, 1)
# print("Shape of y:", y.shape) # (100, 1)
# print("Sample X:", X[:5].flatten())
# print("Sample y:", y[:5].flatten())

# --- 3.2 Разделение Данных на Обучающую и Тестовую Выборки ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
# print(f"\nTrain samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

# --- 3.3 Масштабирование Данных с MinMaxScaler ---
# **Важно:** Масштабируем признаки (X) и целевую переменную (y) ОТДЕЛЬНО,
# так как у них разные диапазоны и распределения.
# Скалеры обучаются ТОЛЬКО на обучающих данных.

# Масштабирование X
scaler_x = MinMaxScaler(feature_range=(0, 1))
# Обучаем на X_train и трансформируем X_train
X_train_scaled = scaler_x.fit_transform(X_train)
# Трансформируем X_test, используя параметры (min/max) от X_train
X_test_scaled = scaler_x.transform(X_test)

# Масштабирование y
scaler_y = MinMaxScaler(feature_range=(0, 1))
# Обучаем на y_train и трансформируем y_train
# reshape(-1, 1) нужен, т.к. скалеры ожидают 2D массив
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
# Трансформируем y_test, используя параметры от y_train
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

# print("\n--- Scaling Results ---")
# print("Original X_train sample:", X_train[:3].flatten())
# print("Scaled X_train sample:", X_train_scaled[:3].flatten())
# print("Original y_train sample:", y_train[:3].flatten())
# print("Scaled y_train sample:", y_train_scaled[:3].flatten())
# print(f"Min/Max of scaled X_train: {X_train_scaled.min():.2f}/{X_train_scaled.max():.2f}")
# print(f"Min/Max of scaled y_train: {y_train_scaled.min():.2f}/{y_train_scaled.max():.2f}")

# --------------------------------------------------

# Блок 4: Обучение Модели SVR

# Создаем экземпляр SVR с линейным ядром
# C - параметр регуляризации (больше C -> меньше регуляризации)
# epsilon - ширина "трубки", где ошибки не штрафуются
svr_linear = SVR(kernel='linear', C=SVR_C, epsilon=SVR_EPSILON)

# print("\nTraining Linear SVR model...")
# Обучаем модель на МАСШТАБИРОВАННЫХ данных
# y_train_scaled.ravel() преобразует y обратно в 1D массив, как ожидает SVR.fit()
svr_linear.fit(X_train_scaled, y_train_scaled.ravel())
# print("Training complete.")

# Вывод параметров (для линейного ядра)
# print(f"SVR Coefficient (slope): {svr_linear.coef_[0][0]:.4f}") # Коэффициент наклона в масштабированном пространстве
# print(f"SVR Intercept: {svr_linear.intercept_[0]:.4f}") # Свободный член в масштабированном пространстве

# --------------------------------------------------

# Блок 5: Предсказание и Обратное Преобразование

# --- 5.1 Предсказание на Масштабированных Тестовых Данных ---
# Модель делает предсказания в том же масштабе, в котором она обучалась ([0, 1])
y_pred_scaled = svr_linear.predict(X_test_scaled)
# print("\nScaled Predictions (first 5):", y_pred_scaled[:5])
# print("Scaled True Test Values (first 5):", y_test_scaled[:5].flatten())

# --- 5.2 Обратное Преобразование Предсказаний ---
# Используем скалер, обученный на **целевой переменной** (`scaler_y`),
# чтобы вернуть предсказания в исходный масштаб.
# reshape(-1, 1) нужен для метода inverse_transform
y_pred_original_scale = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))

# print("\n--- Inverse Transformation ---")
# print("Scaled Predictions (first 5):", y_pred_scaled[:5])
# print("Predictions in Original Scale (first 5):", y_pred_original_scale[:5].flatten())
# print("True Test Values in Original Scale (first 5):", y_test[:5].flatten())

# --------------------------------------------------

# Блок 6: Оценка Модели в Исходном Масштабе

# Сравниваем истинные значения `y_test` (в оригинальном масштабе)
# с предсказанными значениями `y_pred_original_scale` (тоже в оригинальном масштабе).

mse = mean_squared_error(y_test, y_pred_original_scale)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_original_scale)
r2 = r2_score(y_test, y_pred_original_scale)

print("\n--- Model Evaluation (Original Scale) ---")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R2): {r2:.4f}") # Должен быть достаточно высоким для линейных данных

# --------------------------------------------------

# Блок 7: Визуализация Результатов

# plt.figure(figsize=(10, 6))

# # Отображаем исходные тестовые точки
# plt.scatter(X_test, y_test, edgecolor='black', c='cornflowerblue', s=50, label='Actual Test Data')

# # Отображаем линию регрессии SVR
# # Для построения линии используем предсказания в оригинальном масштабе
# # Сортируем X_test для плавной линии (если точек много)
# sort_axis = np.argsort(X_test.ravel())
# plt.plot(X_test[sort_axis], y_pred_original_scale[sort_axis], color='red', linewidth=3, label='SVR Linear Fit')

# # Опционально: Отобразим "трубку" epsilon (в оригинальном масштабе)
# # Нужно преобразовать epsilon из масштаба [0,1] обратно
# # Это приближенно, т.к. epsilon применяется в пространстве признаков/цели после масштабирования
# # Но для визуализации можно масштабировать предсказанную линию
# epsilon_original_scale = SVR_EPSILON * (scaler_y.data_max_ - scaler_y.data_min_) # Приближенная ширина в оригинальном масштабе
# plt.fill_between(X_test[sort_axis].ravel(),
#                  (y_pred_original_scale[sort_axis] - epsilon_original_scale).ravel(),
#                  (y_pred_original_scale[sort_axis] + epsilon_original_scale).ravel(),
#                  color='gray', alpha=0.2, label=f'SVR Epsilon Margin ({SVR_EPSILON:.2f} scaled)')


# plt.title(f'Linear SVR (C={SVR_C}, epsilon={SVR_EPSILON}) with MinMaxScaler')
# plt.xlabel("Feature (X)")
# plt.ylabel("Target (Y)")
# plt.legend()
# plt.grid(True)
# plt.show()

# --- Конец Примера ---

# --------------------------------------------------

# Блок 8: Выводы

# - SVR с `kernel='linear'` может решать задачи линейной регрессии.
# - **Масштабирование признаков (и часто цели) критически важно** для SVR из-за его чувствительности к расстояниям и маржинам. `MinMaxScaler` (или `StandardScaler`) обязателен.
# - Скалеры должны обучаться (`fit`) **только на обучающих данных**.
# - Для оценки модели и интерпретации предсказаний в исходных единицах необходимо выполнить **обратное преобразование** (`inverse_transform`) с помощью скалера, обученного на **целевой переменной**.
# - Гиперпараметры `C` и `epsilon` влияют на поведение SVR и требуют подбора (например, с помощью кросс-валидации).

# --------------------------------------------------